# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [1]:
!pip install pymorphy2
!pip install razdel

     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 8.2 MB 33.8 MB/s 


In [2]:
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from pymorphy2 import MorphAnalyzer
from string import punctuation 
from razdel import tokenize as razdel_tokenize
import spacy


punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
morph = MorphAnalyzer()
nlp = spacy.load('en', disable=['parser', 'ner'])

In [4]:
def normalize(text):
    
    doc = nlp(text)
    
    words = [token.lemma_ for token in doc if not token.is_punct]

    return words

In [5]:
def compute_overlap(signature, context):
  overlap = set(signature) & set(context)
  return len(overlap)

In [26]:
def lesk(word, sentence):
  """Ваш код тут"""
  bestsense = 0
  maxoverlap = 0
  
  context = normalize(sentence)
  if word in context:
    context = context[:context.index(word)] + context[context.index(word)+1:]

  for i, syns in enumerate(wn.synsets(word)):
    # ваш код 
    signature = normalize(syns.definition())
    overlap = compute_overlap(signature, context)
    
    if overlap > maxoverlap:
      maxoverlap = overlap
      bestsense = i

  return bestsense


**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [7]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [8]:
disambig_corpus = []

for sentence in corpus_wsd:
  for word in sentence:
    if word[0] != '':
      disambig_corpus.append(word)

In [9]:
sentences = []

for sent in corpus:
  cnt = 0
  for word in sent.split('\n'):

    if word.split('\t')[0] != '':
      cnt += 1
  
  for i in range(cnt):
    sentences.append(' '.join([s.split('\t')[2] for s in sent.split('\n')]))

In [10]:
from tqdm import tqdm

In [27]:
result = [wn.synsets(word[1])[lesk(word[1], sentence)] == wn.lemma_from_key(word[0]).synset() for sentence, word in zip(sentences[:10000], disambig_corpus[:10000])]

In [28]:
result.count(True)/len(result)

0.3737

Как можно видеть, результат работы алгоритма не совсем хороший. В ходе написания алгоритма было обнаружено, что лемматизация очень сильно влияет на качество алгоритма. Из всех библиотек, на мой взгляд, лучшим образом показал себя SpaCy.

### Задание 2* (2 балла)

In [13]:
!gdown --id 1DIRIYCkUBSk6lb0RTQhF7bTs-lKKkPSF

Downloading...
From: https://drive.google.com/uc?id=1DIRIYCkUBSk6lb0RTQhF7bTs-lKKkPSF
To: /content/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
100% 1.46G/1.46G [00:12<00:00, 121MB/s]


In [14]:
!wget https://raw.githubusercontent.com/nlpub/russe-wsi-kit/master/data/main/wiki-wiki/train.csv

--2022-02-04 15:20:48--  https://raw.githubusercontent.com/nlpub/russe-wsi-kit/master/data/main/wiki-wiki/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 365614 (357K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 357.04K  --.-KB/s    in 0.004s  

2022-02-04 15:20:48 (92.1 MB/s) - ‘train.csv’ saved [365614/365614]



In [15]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-wc76_19q
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-wc76_19q
  Created wheel for adagram: filename=adagram-0.0.1-cp37-cp37m-linux_x86_64.whl size=450487 sha256=419b6b1948a77d9b8b090e6315804ab4e93eb3ca57865a229bee434b83bcf840
  Stored in directory: /tmp/pip-ephem-wheel-cache-_a83siiy/wheels/5a/8c/f9/7dee902dd325a3317e768f126aa6f7aa085c79a6e763ed2cb8
Successfully built adagram


In [16]:
import pandas as pd
import adagram
import numpy as np
from sklearn.metrics import adjusted_rand_score

In [17]:
df = pd.read_csv('train.csv', sep='\t')

In [18]:
def get_embedding_adagram(text, model, window, dim):
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 

        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [19]:
def normalize_rus(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [20]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [21]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [22]:
dim = 300
window = 5

In [23]:
from sklearn.cluster import *

In [29]:
models = [AffinityPropagation(damping=0.7, max_iter=100, verbose=True), KMeans(3, tol=0.001, max_iter=350), DBSCAN(min_samples=1, eps=0.1,leaf_size=25)]
ARI_by_model = {type(model).__name__: [] for model in models}

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize_rus)
    X = np.zeros((len(texts), dim))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding_adagram(text, vm, window, dim)
        
    for model in models:
      model.fit(X)
      labels = np.array(model.labels_)+1
      ARI_by_model[type(model).__name__].append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

/usr/local/lib/python3.7/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


Converged after 32 iterations.


/usr/local/lib/python3.7/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


Did not converge


/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_affinity_propagation.py:253: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


Did not converge


/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_affinity_propagation.py:253: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


Converged after 63 iterations.


In [30]:
{name: np.mean(val) for name, val in ARI_by_model.items()}

{'AffinityPropagation': 0.06708180628651,
 'DBSCAN': 0.0010826021180129009,
 'KMeans': 0.3570428185789412}

Среди всех алгоритмов лучшее качество достигается KMeans. Причём при каждом запуске его результаты могут сильно разниться.